# Word2Vec model

## Set up

In [1]:
import numpy as np 
import pandas as pd
import nltk
from nltk.corpus import stopwords
import re

## Load dataset

In [2]:
df = pd.read_csv(r'../resources/IMDB Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


## Tiền xử lý dữ liệu

- Loại bỏ các thẻ HTML
- Loại bỏ các kí tự đặc biệt 
- Chuyển chữ hoa thành chữ thường 
- Loại bỏ stopwords

### Loại bỏ thẻ HTML

In [3]:
df['review'] = df['review'].str.replace("<br />", "")
df['review'][2]

'I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.This was the most I\'d laughed at one of Woody\'s comedies in years (dare I say a decade?). While I\'ve never been impressed with Scarlet Johanson, in this she managed to tone down her "sexy" image and jumped right into a average, but spirited young woman.This may not be the crown jewel of his career, but it was wittier than "Devil Wears Prada" and more interesting than "Superman" a great comedy to go see with friends.'

### Loại bỏ các kí tự đặc biệt và khoảng trắng thừa

In [4]:
# Hàm để chuyển các ký tự đặc biệt thành khoảng trắng và loại bỏ khoảng trắng thừa
def remove_punctuation(text):
    # Chuyển các ký tự đặc biệt thành khoảng trắng
    text = re.sub("n\'t", " not", text)
    text = re.sub("\'s", " is", text)
    text = re.sub("\'re", " are", text)
    text = re.sub("\'m", " am", text)
    text = re.sub(r'[^\w\s]', ' ', text)
    # Loại bỏ các khoảng trắng thừa
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [5]:
df['review'] = df['review'].apply(remove_punctuation)

### Chuyển chữ hoa thành chữ thường

In [7]:
df['review'] = df['review'].str.lower()

### Loại bỏ stopwords

In [8]:
# Tải stop words từ nltk
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
stop_words.remove('no')
stop_words.remove('not')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PAVT\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
# import pickle
# with open("../resources/stopwords.pkl", "wb") as f:
#     pickle.dump(stop_words, f)

In [9]:
def remove_stop_words(text):
    words = text.split()
    filter_words = [word for word in words if word not in stop_words]
    return ' '.join(filter_words)

In [10]:
df['review'] = df['review'].apply(remove_stop_words)

## Chia dữ liệu

In [11]:
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['review'], df['sentiment'], test_size=0.3, random_state=42)

## Word2Vec model

In [6]:
from gensim.models import Word2Vec

### Word embeddings

In [12]:
sentences = []
for sentence in X_train:
    sentences.append(sentence.split())

In [12]:
test_sentences = []
for sentence in X_test:
    test_sentences.append(sentence.split())

In [13]:
import pickle
with open("../resources/test_sentences.pkl", "wb") as f:
    pickle.dump(test_sentences,f)

In [30]:
# import pickle
# with open("../resources/sentences.pkl", "wb") as f:
#     pickle.dump(sentences,f)

In [13]:
model = Word2Vec(sentences, vector_size=200, window=5, sg=1, min_count=5, workers = 80)
#model = Word2Vec.load("../resources/word_embeddings.model")

In [5]:
word_embeddings = [ [model.wv[word] for word in sent if word in model.wv ] for sent in sentences ]

__Save model__

In [36]:
import pickle 
with open("../resources/sentences.pkl", "wb") as f:
    pickle.dump(sentences, f)

In [16]:
model.save("../resources/word_embeddings.model")

### Sentence embeddings

In [14]:
def sum_weights(vectors, w):
    return np.matmul(w.T, vectors)

In [15]:
def softmax(arr):
    sum_exp = np.sum(np.exp(arr))
    weights = np.zeros(len(arr))
    for i in range(len(arr)):
        weights[i] = np.exp(arr[i])/sum_exp
    return weights

#### Các tiêu chí đánh giá phim: 
- Kịch bản: 
    + coherent/ incoherent
    + unpredictable/ predictable
- Ý nghĩa phim: meaningful/ meaningless
- Hiệu ứng: impressive / unimpressive
- Cảnh quay: heartfelt / insincere

In [34]:
list_positive_words = ['heartfelt', 'gripping', 'impressive', 'meaningful', 'coherent']
list_negative_words = ['insincere', 'predictable', 'soporific', 'illogical', 'uninteresting']

# incoherent == uninteresting
# meaningless == illogical
# unimpressive == soporific
# unpredictable == gripping

#### Xây dựng vector tổng hợp của các từ tích cực và tiêu cực

In [35]:
def normalize(v):
    return v/np.linalg.norm(v)

In [36]:
list_positive = np.array([ normalize(model.wv[word]) for word in list_positive_words ])
list_negative = np.array([ normalize(model.wv[word]) for word in list_negative_words ])

In [37]:
positive_vector = normalize( sum_weights(list_positive, np.array([1,2,1,2,1]) ) )
negative_vector = normalize( sum_weights(list_negative, np.array([1,2,1,2,1]) ) )

In [38]:
def sentence_to_vector(sentence, model):
    word_vectors = np.array([model.wv[word] for word in sentence if word in model.wv]) # size T*vector_size
    pos_weights = []
    neg_weights = []
    res = np.zeros(model.vector_size*2)
    
    if len(word_vectors) == 0:
        return res
    pos_weights = np.matmul(word_vectors, positive_vector.T)
    neg_weights = np.matmul(word_vectors, negative_vector.T)
    pos_res = sum_weights(word_vectors, softmax(pos_weights) )
    neg_res = sum_weights(word_vectors, softmax(neg_weights) )
    return np.concatenate([pos_res, neg_res])

### Xây dựng tập huấn luyện và tập kiểm tra

In [39]:
X_w2v_train = [ sentence_to_vector(sent.split(), model)  for sent in X_train]
X_w2v_test =  [ sentence_to_vector(sent.split(), model)  for sent in X_test ]

In [41]:
import pickle
with open("../resources/X_w2v_train.pkl", 'wb') as f:
    pickle.dump(X_w2v_train, f)
with open("../resources/X_w2v_test.pkl", 'wb') as f:
    pickle.dump(X_w2v_test, f)
with open("../resources/y_train.pkl", 'wb') as f:
    pickle.dump(y_train, f)
with open("../resources/y_test.pkl", 'wb') as f:
    pickle.dump(y_test, f)

In [40]:
for i in range(len(X_w2v_train)):
    if(X_w2v_train[i].shape != (400,) ):
        print(i)